In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [2]:
import sklearn
sklearn.show_versions()


System:
    python: 3.8.5 (default, Sep  4 2020, 02:22:02)  [Clang 10.0.0 ]
executable: /Users/metra4ok/anaconda3/bin/python
   machine: macOS-10.15.7-x86_64-i386-64bit

Python dependencies:
          pip: 20.3.3
   setuptools: 52.0.0.post20210125
      sklearn: 0.24.1
        numpy: 1.19.2
        scipy: 1.5.2
       Cython: 0.29.21
       pandas: 1.2.1
   matplotlib: 3.3.2
       joblib: 1.0.0
threadpoolctl: 2.1.0

Built with OpenMP: True


In [3]:
import random

seed = 42
np.random.seed(seed)
random.seed(seed)

# Загрузка датасета

In [4]:
data = pd.read_csv('preprocessed_data.csv', index_col=0)

data.head()

,МАРКА_rare,МАРКА_Э76ХФ,МАРКА_Э90ХАФ,ПРОФИЛЬ_rare,ПРОФИЛЬ_Р65,t вып-обр,t обработка,t под током,t продувка,ПСН гр.,...,химшлак первый MnO_1,химшлак первый R_1,химшлак первый SiO2_1,химшлак последний Al2O3,химшлак последний CaO,химшлак последний FeO,химшлак последний MgO,химшлак последний MnO,химшлак последний R,химшлак последний SiO2
0,0,1,0,0,1,29.0,45.366667,24.400000,41.033333,3.004414,...,0.12,2.6,22.704914,NaN,53.4,1.0,5.8,0.15,2.6,NaN
1,0,1,0,0,1,26.0,44.066667,13.866667,44.066667,3.004414,...,0.14,2.2,22.704914,NaN,59.3,0.6,4.1,0.11,2.3,NaN
2,0,1,0,0,1,24.0,43.350000,17.950000,43.350000,3.004414,...,0.12,2.6,22.704914,NaN,57.2,0.5,8.1,0.13,2.3,NaN
3,0,1,0,0,1,17.0,46.183333,19.816667,46.183333,3.004414,...,0.11,2.4,22.704914,NaN,60.3,0.5,6.3,0.12,2.6,NaN
4,0,1,0,0,1,20.0,48.500000,17.033333,48.500000,3.004414,...,0.12,2.4,22.704914,NaN,60.3,0.6,4.5,0.09,2.5,NaN


In [5]:
data.shape

(6237, 72)

In [7]:
target_features = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний FeO', 'химшлак последний MgO',
                   'химшлак последний MnO', 'химшлак последний R',
                   'химшлак последний SiO2']

categorical_features = ['МАРКА_rare', 'МАРКА_Э76ХФ', 'МАРКА_Э90ХАФ',
                        'ПРОФИЛЬ_rare', 'ПРОФИЛЬ_Р65']

numerical_features = ['t вып-обр', 't обработка', 't под током', 't продувка',
       'ПСН гр.', 'чист расход C', 'чист расход Cr', 'чист расход Mn',
       'чист расход Si', 'чист расход V', 'температура первая',
       'температура последняя', 'Ar (интенс.)', 'эл. энергия (интенс.)',
       'произв жидкая сталь', 'расход газ Ar', 'сыпуч известь РП',
       'сыпуч кокс пыль УСТК', 'сыпуч  кокс. мелочь КМ1', 'сыпуч  шпат плав.',
       'ферспл CaC2', 'ферспл  FeV-80', 'ферспл  Mn5Si65Al0.5',
       'ферспл SiMn18', 'ферспл  фх850А', 'эл. энергия',
       'химсталь первый Al_1', 'химсталь первый C_1', 'химсталь первый Cr_1',
       'химсталь первый Cu_1', 'химсталь первый Mn_1', 'химсталь первый Mo_1',
       'химсталь первый N_1', 'химсталь первый Ni_1', 'химсталь первый P_1',
       'химсталь первый S_1', 'химсталь первый Si_1', 'химсталь первый Ti_1',
       'химсталь первый V_1', 'химсталь последний Al', 'химсталь последний C',
       'химсталь последний Ca', 'химсталь последний Cr',
       'химсталь последний Cu', 'химсталь последний Mn',
       'химсталь последний Mo', 'химсталь последний N',
       'химсталь последний Ni', 'химсталь последний P', 'химсталь последний S',
       'химсталь последний Si', 'химсталь последний Ti',
       'химсталь последний V', 'химшлак первый Al2O3_1',
       'химшлак первый CaO_1', 'химшлак первый FeO_1', 'химшлак первый MgO_1',
       'химшлак первый MnO_1', 'химшлак первый R_1', 'химшлак первый SiO2_1']

Для каждого таргета будем строить свою модель.

**У некоторых таргетов есть пропуски значений - их необходимо удалять**

In [19]:
scores = {}
models = {}

# Al2O3

Значение MAPE на бейзлайне = 11.38%  
В результате здесь **MAPE = 11.14%**

In [9]:
target = 'химшлак последний Al2O3'
targets_to_drop = ['химшлак последний CaO', 'химшлак последний FeO',
                   'химшлак последний MgO', 'химшлак последний MnO', 'химшлак последний R',
                   'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [10]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Отбор фич с помощью Lasso

In [11]:
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=seed))
sel_.fit(X, y)

selected_features = X.columns[(sel_.get_support())]

print('Было признаков: {}'.format((X.shape[1])))
print('Отобрано признаков: {}'.format(len(selected_features)))
print('Удалено признаков: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

Было признаков: 65
Отобрано признаков: 38
Удалено признаков: 27


In [12]:
X = X[selected_features]

## Модель со стекингом

In [18]:
estimators = [('lr', LinearRegression()), 
              ('lgb', LGBMRegressor()),
              ('rf', RandomForestRegressor())]

stacking_reg = StackingRegressor(estimators=estimators)
scores_stacking = cross_val_score(stacking_reg, X, y, 
                                  scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_stacking.mean())*100))

Кросс-валидация: 11.14420274313871%


In [20]:
scores['Al2O3'] = np.abs(scores_stacking.mean())*100
models['Al2O3'] = stacking_reg

# CaO

MAPE на бейзлайне = 4.82%  
В итоговом пайплайне **MAPE = 4.76%**

In [22]:
target = 'химшлак последний CaO'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний FeO',
                   'химшлак последний MgO', 'химшлак последний MnO',
                   'химшлак последний R', 'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [23]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Отбор фич с помощью Lasso

In [24]:
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=seed))
sel_.fit(X, y)

selected_features = X.columns[(sel_.get_support())]

print('Было признаков: {}'.format((X.shape[1])))
print('Отобрано признаков: {}'.format(len(selected_features)))
print('Удалено признаков: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

X = X[selected_features]

Было признаков: 65
Отобрано признаков: 50
Удалено признаков: 15


In [26]:
estimators = [('lr', LinearRegression()), 
              ('lgb', LGBMRegressor()),
              ('rf', RandomForestRegressor())]

stacking_reg = StackingRegressor(estimators=estimators)
scores_stacking = cross_val_score(stacking_reg, X, y, 
                                  scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_stacking.mean())*100))

Кросс-валидация: 4.76%


In [27]:
scores['CaO'] = np.abs(scores_stacking.mean())*100
models['CaO'] = stacking_reg

# FeO

Здесь все сложно, улучшать не очень получается.  
MAPE на бейзлайне = 35.29%  
В итоговом пайплайне **MAPE = 35.09%**

In [28]:
target = 'химшлак последний FeO'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний MgO', 'химшлак последний MnO',
                   'химшлак последний R', 'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [29]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Отбор фич

In [30]:
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=seed))
sel_.fit(X, y)

selected_features = X.columns[(sel_.get_support())]

print('Было признаков: {}'.format((X.shape[1])))
print('Отобрано признаков: {}'.format(len(selected_features)))
print('Удалено признаков: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

X = X[selected_features]

Было признаков: 65
Отобрано признаков: 16
Удалено признаков: 49


In [32]:
estimators = [('lr', LinearRegression()), 
              ('lgb', LGBMRegressor()),
              ('rf', RandomForestRegressor())]

stacking_reg = StackingRegressor(estimators=estimators)
scores_stacking = cross_val_score(stacking_reg, X, y, 
                                  scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_stacking.mean())*100))

Кросс-валидация: 35.10%


In [33]:
scores['FeO'] = np.abs(scores_stacking.mean())*100
models['FeO'] = stacking_reg

# MgO

MAPE на бейзлайне = 26.21%%  
В итоговом пайплайне **MAPE = 25.73%**

In [34]:
target = 'химшлак последний MgO'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний FeO', 'химшлак последний MnO',
                   'химшлак последний R', 'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [35]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Отбор фич

In [36]:
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=seed))
sel_.fit(X, y)

selected_features = X.columns[(sel_.get_support())]

print('Было признаков: {}'.format((X.shape[1])))
print('Отобрано признаков: {}'.format(len(selected_features)))
print('Удалено признаков: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

X = X[selected_features]

Было признаков: 65
Отобрано признаков: 48
Удалено признаков: 17


## Стекинг

In [37]:
estimators = [('lr', LinearRegression()), 
              ('lgb', LGBMRegressor()),
              ('rf', RandomForestRegressor())]

stacking_reg = StackingRegressor(estimators=estimators)
scores_stacking = cross_val_score(stacking_reg, X, y, 
                                  scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_stacking.mean())*100))

Кросс-валидация: 25.73%


In [38]:
scores['MgO'] = np.abs(scores_stacking.mean())*100
models['MgO'] = stacking_reg

# MnO

**MAPE на бейзлайне = 30.95%**. Преодолеть не удалось.

In [56]:
target = 'химшлак последний MnO'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний FeO', 'химшлак последний MgO',
                   'химшлак последний R', 'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [57]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Линрег

In [58]:
lin_reg_base = LinearRegression()
scores_lin_reg_base = cross_val_score(lin_reg_base, X, y, 
                                      scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_lin_reg_base.mean())*100))

Кросс-валидация: 30.95%


In [59]:
scores['MnO'] = np.abs(scores_lin_reg_base.mean())*100
models['MnO'] = lin_reg_base

# R

MAPE на бейзлайне = 6.19%   
В итоговом пайплайне **MAPE = 5.98%**

In [66]:
target = 'химшлак последний R'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний FeO', 'химшлак последний MgO',
                   'химшлак последний MnO', 'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [67]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Отбор фич

In [68]:
sel_ = SelectFromModel(Lasso(alpha=0.001, random_state=seed))
sel_.fit(X, y)

selected_features = X.columns[(sel_.get_support())]

print('Было признаков: {}'.format((X.shape[1])))
print('Отобрано признаков: {}'.format(len(selected_features)))
print('Удалено признаков: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

X = X[selected_features]

Было признаков: 65
Отобрано признаков: 39
Удалено признаков: 26


## Стекинг

In [69]:
estimators = [('lr', LinearRegression()), 
              ('lgb', LGBMRegressor()),
              ('rf', RandomForestRegressor())]

stacking_reg = StackingRegressor(estimators=estimators)
scores_stacking = cross_val_score(stacking_reg, X, y, 
                                  scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_stacking.mean())*100))

Кросс-валидация: 5.98%


In [70]:
scores['R'] = np.abs(scores_stacking.mean())*100
models['R'] = stacking_reg

# SiO2

**MAPE на бейзлайне = 6.98%**. Повысить не удалось.

In [90]:
target = 'химшлак последний SiO2'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний FeO', 'химшлак последний MgO',
                   'химшлак последний MnO', 'химшлак последний R']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

## Нормализация

In [91]:
X_categorical = pd.DataFrame(np.array(X[categorical_features]), columns=categorical_features)
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X.drop(categorical_features, axis=1)), 
                        columns=numerical_features)

X = X_categorical.join(X_scaled)

## Линрег

In [92]:
lin_reg_base = LinearRegression()
scores_lin_reg_base = cross_val_score(lin_reg_base, X, y, 
                                      scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация: {:.2f}%".format(np.abs(scores_lin_reg_base.mean())*100))

Кросс-валидация: 6.98%


In [93]:
scores['SiO2'] = np.abs(scores_lin_reg_base.mean())*100
models['SiO2'] = lin_reg_base

# Выводы

In [98]:
pd.DataFrame.from_dict(scores, orient='index', columns=['MAPE score %'])

,MAPE score %
Al2O3,11.144203
CaO,4.764596
FeO,35.100772
MgO,25.726852
MnO,30.952955
R,5.980660
SiO2,6.977944


В результате можно сделать следующие выводы по данному этапу:

- Целевые переменные **'химшлак последний CaO'**, **'химшлак последний R'** и **'химшлак последний SiO2'** предсказываются с достаточно высокой точностью.
- Целевая переменная **'химшлак последний Al2O3'** имеет погрешность предсказания 11.14%, что вполне удовлетворительно, однако следует внимательнее проработать модель, возможно метрику удастся улучшить.
- Целевые переменные **'химшлак последний FeO'**, **'химшлак последний MgO'** и **'химшлак последний MnO'** предсказываются с довольно высокой погрешностью, которую не получается снизить до приемлемого уровня. Возможно, по имеющимся данным в принципе невозможно обеспечить низкую ошибку.